In [27]:
import pandas as pd

from dash import Dash, dcc, html
from dash.dependencies import Input, Output

spacex_df = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
)

# Defensive cleanup (helps avoid weird dropdown duplicates)
spacex_df["Launch Site"] = spacex_df["Launch Site"].astype(str).str.strip()

In [28]:
launch_sites = sorted(spacex_df["Launch Site"].unique())

site_options = [{"label": "All Sites", "value": "ALL"}] + [
    {"label": site, "value": site} for site in launch_sites
]


In [29]:
app = Dash(__name__)

min_payload = 0
max_payload = 10000

app.layout = html.Div(
    children=[
        html.H1(
            "SpaceX Launch Records Dashboard",
            style={"textAlign": "center"}
        ),

        # Site dropdown
        dcc.Dropdown(
            id="site-dropdown",
            options=site_options,
            value="ALL",
            placeholder="Select a Launch Site here",
            searchable=True,
            clearable=False
        ),

        html.Br(),

        # Payload slider (applies to scatter, harmless for pie)
        html.P("Payload range (Kg):"),
        dcc.RangeSlider(
            id="payload-slider",
            min=min_payload,
            max=max_payload,
            step=1000,
            value=[min_payload, max_payload],
            marks={
                0: "0",
                2500: "2500",
                5000: "5000",
                7500: "7500",
                10000: "10000"
            }
        ),

        html.Br(),
        html.Hr(),

        # Tabs for the two visuals
        dcc.Tabs(
            value="tab-pie",
            children=[
                dcc.Tab(
                    label="Launch Success Overview",
                    value="tab-pie",
                    children=[
                        dcc.Graph(id="success-pie-chart", style={"height": "650px"})
                    ],
                ),
                dcc.Tab(
                    label="Payload vs Success",
                    value="tab-scatter",
                    children=[
                        dcc.Graph(id="success-payload-scatter-chart", style={"height": "650px"})
                    ],
                ),
            ],
        ),
    ]
)

In [30]:
### app.run(mode="inline", debug=True)


In [31]:
import plotly.express as px

In [32]:
@app.callback(
    Output("success-pie-chart", "figure"),
    Input("site-dropdown", "value")
)
def update_pie_chart(selected_site):
    if selected_site == "ALL":
        # Total successful launches by site
        success_by_site = (
            spacex_df[spacex_df["class"] == 1]
            .groupby("Launch Site", as_index=False)
            .size()
            .rename(columns={"size": "Successes"})
        )

        fig = px.pie(
            success_by_site,
            values="Successes",
            names="Launch Site",
            title="Total Success Launches By Site"
        )
        return fig

  # Success vs Failure for a specific site
    site_df = spacex_df[spacex_df["Launch Site"] == selected_site]
    outcome_counts = (
        site_df["class"]
        .value_counts()
        .rename_axis("class")
        .reset_index(name="count")
    )

    # Make labels readable
    outcome_counts["Outcome"] = outcome_counts["class"].map({1: "Success", 0: "Failure"})

    fig = px.pie(
        outcome_counts,
        values="count",
        names="Outcome",
        title=f"Success vs Failure for site {selected_site}"
    )
    return fig

In [33]:
#app.run(mode="inline", debug=False)

In [34]:
@app.callback(
    Output("success-payload-scatter-chart", "figure"),
    [
        Input("site-dropdown", "value"),
        Input("payload-slider", "value")
    ]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range

    # Filter by payload range first
    filtered = spacex_df[
        (spacex_df["Payload Mass (kg)"] >= low) &
        (spacex_df["Payload Mass (kg)"] <= high)
    ]

    # Then filter by site if needed
    if selected_site != "ALL":
        filtered = filtered[filtered["Launch Site"] == selected_site]
        title = f"Correlation between Payload and Success for {selected_site}"
    else:
        title = "Correlation between Payload and Success for all Sites"

    fig = px.scatter(
        filtered,
        x="Payload Mass (kg)",
        y="class",
        color="Booster Version Category",
        title=title
    )

    return fig

In [35]:
app.run(mode="inline", debug=False)